In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Libraries and functions

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
# import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
from os import path
import roamability as rb
import re

downloads = 'C:/Users/balob/Downloads'

In [99]:
%reset -f

# Examples

### MS SQL request

In [208]:
sql_srt='SELECT COUNT(*) FROM ACCOUNTS;'
with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### DMI Oracle request

In [211]:
sql_srt = 'SELECT * FROM SPONSOR'
# with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI_TEST') as cnxn:
with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

### BSS MySql request

In [1096]:
sql_srt='SELECT MSISDN, VisitedNetworkTadig FROM TAP.GPRS_CALL LIMIT 5'
with rb.MySqlConnect('172.18.11.40', 'BSS', 'noc', 'WcQUzkXiXwoxnFfGnRxb') as cnxn:
    df = pd.read_sql_query(sql_srt, cnxn)
df

,MSISDN,VisitedNetworkTadig
0,447700095423,BRAV1
1,972541033641,BRAV1
2,972541039874,BRAV1
3,972541033641,BRAV1
4,447700095423,BRAV1


### Soap API OCS Portal

In [457]:
# %reset -f
from zeep import Client
from zeep.wsse.username import UsernameToken

connect = input("Choose connection (prod or lab):")

if connect == 'lab':
    user = 'lab_api@roamability.com'
    user = 'api_lab@dialoq.com'
    password = '25D55AD283AA400AF464C76D713C07AD' #Lab
    password = 'FAC4EC08B5FC0CCF8145BB79E8F24F47' #Lab
    api_link = 'https://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
    api_link = 'https://labocsapi.roamability.com:6443/ocsapi/roamability/api/roamability.wsdl'
    api_link = 'http://172.20.39.7:8585/ocsapi/roamability/api/roamability.wsdl' #Lab
elif connect == 'prod':
    user = 'prod_api@roamability.com'
    password = '25D55AD283AA400AF464C76D713C07AD'
    api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'https://ocsapi.roamability.com:7443/ocsapi/roamability/api/'
#     api_link = 'https://ocsapi2.roamability.com:7443/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.10:8585/ocsapi/roamability/api/roamability.wsdl'
#     api_link = 'http://172.20.35.12:8585/ocsapi/roamability/api/roamability.wsdl'

user_name_token = UsernameToken(user, password)
user_name_token.use_digest = True
client = Client(api_link, wsse=user_name_token)

reseller = client.service.getResellerInfo()
accounts = client.service.getAccounts()

if reseller.result.code == '1':
    print('You are connected to Reseller: {}.\
    \nAvailable credit for the Reseller: {:.0f} USD.\n'.\
    format(reseller.reseller.resellerName,reseller.reseller.availableCredit))
else:
    print('Error in getting Reseller info: {}'.format(reseller.result.description))

if accounts.result.code == '1':
    accounts_dict = {account.accountId:account.accountName for account in accounts.accounts.accounts}
    print('The Reseller contains the following accounts:')
    for i,[account_id,account_name] in enumerate(accounts_dict.items()):
        print(' {:<5}{} (accountID: {})'.format(i,account_name,account_id))
else:
    print('Error in getting Account info: {}'.format(accounts.result.description))

# Subscriber in lab in STI account
#subscriber_id = client.service.getSubscriberById('934420')
#subscriber_iccid = client.service.getSubscriberByICCID('8997219121000031446')
# client.service.getSubscriberPrePaidPackages('831882', 1)
#client.service.getSubscriberByIMSI('425019613023160')
#client.service.getSubscriberById('209287')
#client.service.getSubscriberByICCID('8997219121000022660')
# client.service.assignSubscriberPrePaidPackage('831882', 1, dt.datetime.strptime('2019-07-07','%Y-%m-%d'))
# client.service.getSubscriberPrePaidPackages('937939', 1)
# client.service.expireSubscriberPrePaidPackage(1848972,60311)
# client.service.getSubscriberPrePaidPackages(1848972,1)
# prices = client.service.getMobilePrices() # prices.destinations[0]
# client.service.getPricingPlans()
# client.service.modifySubscriberSpeedLimit(1449606, '256kbps')
# client.service.modifySubscriberSpeedLimit(210000, '1024kbps') # lab
#subscriber_id

Choose connection (prod or lab):lab
You are connected to Reseller: Genesisclub RT.    
Available credit for the Reseller: 2 USD.

The Reseller contains the following accounts:
 0    Genesisclub Test sim cards (accountID: 352475)


In [301]:
# client.service.getSubscriberById(934420)
# client.service.modifySubscriberSpeedLimit(934420, '1024kbps') # lab

### Send MSU

In [230]:
from bs4 import BeautifulSoup
import roamability as rb
#reload(rb)

# add_imsi = '02700000370d00210000bfff010000000000007056d7a52e5fcef694ab2c069c00df34b084853aa0a2bf02195723f978ba96ba8b3a4a28b4df531cfe'
# get_info    = "24027000001F0D00010000BFFF01000000000000102F978372C730834EF445172BD26B8C8F"
# get_info    = "027000001F0D00010000BFFF01000000000000102F978372C730834EF445172BD26B8C8F"
# get_info_mo = "027000001F0D00210000BFFF01000000000000102F978372C730834EF445172BD26B8C8F"
# switch_01_payload = '027000001f0d00010000bfff0100000000000010554f212dbbad49a1caf2573e1a626139'
# switch_02_payload = '027000001F0D00010000BFFF0100000000000040a740AD174703383798A7A6545BB36491'

switch_01_payload = '24027000001f0d00010000bfff0100000000000010554f212dbbad49a1caf2573e1a626139'
switch_02_payload = '24027000001F0D00010000BFFF0100000000000040a740AD174703383798A7A6545BB36491'

#515030190000000
#1d46acf860780
add_imsi_03 = '3C02700000370d00210000bfff0100000000000040939360c23f417f147557b6940c91ba8b7e54be1c8c950a6ed1235ea8e3d70e9c2d7c34876abd68ec'


ogt = {'p4':'48790993070',
       'partner':'97254120624',
       'sure':'447797706411',
       'tot':'66893773228',
       'porto_seguro':'550549900000',
       'rusec_rus':'79028710069',
       'rusec_int':'417999880000024',
       'multi_byte_sponsor':'85263347864',
       'smart1':'639180009880',
       'smart2':'639180009881',
       'smart3':'639180009882',
       'smart4':'639180009883',
       'telzar':'972559900040',
       'x2one':'972553316228', #972553316240 -972553316245
       'cellact':'972557016315',
       'netmore':'46731726312',
       'p4_naka_01':'48790998145',
       'p4_naka_02':'48790998146',
       'p4_naka_03':'48790998147',
       'partner_naka_01':'97254120634',
       'partner_naka_02':'97254120635',
       'partner_naka_03':'97254120636',
       'partner_naka_04':'97254120637',
       'mb':'852633477591',
       'maxcom':'525575709019',
       'orange':'48507909001',
       'jt':'447797707084',
       'tcom':'359999320032',
       'volna':'79785569998',
       'surf_test':'550170002133',
       'surf_prod':'550170000133',
       'Volna':'79785750017'}

rb.greetings_func()
#rb.print_imsi_prof(ogt, imsi, msc, 'as_resp') #'as_mo'
#soup = BeautifulSoup(rb.sri4sm(ogt, msisdn),'xml') #250359000007598 79028710043
#soup = BeautifulSoup(rb.prn(ogt, dgt, imsi),'xml')
#soup = BeautifulSoup(rb.sai(ogt, dgt, imsi, node),'xml') # 0 - vlr; 16 - mme
# soup = BeautifulSoup(rb.sendSMS(ogt, imsi, msc, rb.get_info),'xml')
#soup = BeautifulSoup(rb.cl(ogt, dgt, imsi, d_ssn=7),'xml')
# rb.print_imsi_prof('48790993070', '260060148993323', '46708000031', 'as_resp') #'as_mo' 'as_resp'
# soup = BeautifulSoup(sendSMS(ogt['orange'], '260036610000004', '79639743781', rb.get_info),'xml')
# rb.decode_payload_response(soup.response.text)
# soup = BeautifulSoup(psi('97254120624', '425120000003499', '972571209920', add_imsi),'xml') #250359000007598 79028710043
#soup

Greetings from Roamability!!! 



In [575]:
# soup = BeautifulSoup(rb.cl('48790993070', '46708000031', '260060148993323', d_ssn=7),'xml')
soup = BeautifulSoup(rb.cl('97254120624', '447785012630', '425019610499426', d_ssn=149),'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>0</result>
<src_gt>447785012630</src_gt>
</response>

In [33]:
# reload(rb)
s = '00350abfff0100000000000000fffffff00001070052f0200000003a155608057823070008495210693100112901082906601094001129'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070232F4010000003A95940448388403000849528100020000500108495210690210620102082906601004803854'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF0000107020000000100033A75350738900608000849528100020000600108495210690210139702082906601004901438'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070025F0510000003A55130828920808000859025111106202550108495210690210749802082906601004114330'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070052F0200000008A56780943745100000849521069310050630108290660109400506303085951301009000000'
s = r'00077FF644027100003F0ABFFF0100000000000000FFFFFFF00001070252F0100000008A56780943745100000849521069310050630108290660109400506302084945302072039929'
s = r'00530abfff0100000000000000fffffff00001070115f5200000003a353009591638050008495291211011038501084952106992583165020829066010041024220308494530207203001104085951301009000020'
s = r'003f0abfff0100000000000000fffffff0000107011300140000003a45920509893807000849521069314098990108290660109440479902085951301009000090'
s = r'002b0abfff01000000000000000000001e000107003304200000008a6658079451250200083943111002000098'
s = r'002b0abfff01000000000000000000001d000107003304200000003a7525166054250700083943111002000031'
s = r'002b0abfff010000000000000000000049000107003304200000003a7525166054250700083943111002000031'
rb.decode_payload_response(s)

This SIM card contains the following IMSIs:
Slot 0 334110120000013


In [1082]:
import http
import re
from bs4 import BeautifulSoup
import sys

ss7_url = "172.18.11.10"
ss7_path = "/cgi-bin/ss7gw.fcgi"

S6A_URL = '172.18.11.90'
S6A_PATH = '/cgi-bin/http.fcgi'

def executeHTTP(request, url, path):
    client = http.client.HTTPConnection(url)
    client.request("POST", path, request, {"Content-Type": "text/xml"})
    resp = client.getresponse()
    return resp.read()

def sri4sm(ogt, msisdn, dgt):
    req = """<?xml version=\"1.0\"?>
        <ss7gw request=\"SRI_SM\">
        <d_ssn>6</d_ssn>
        <o_ssn>8</o_ssn>
        <sccp_np>7</sccp_np>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <msisdn>%s</msisdn>
        <priority>1</priority>
        <address>%s</address>
        </ss7gw>""" % (ogt, dgt, msisdn, ogt)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

def sendSMS(ogt, imsi, msc, payload):
    req = """<ss7gw request=\"MTSMS_OTA\">
        <d_ssn>8</d_ssn>
        <o_ssn>8</o_ssn>
        <o_gt>%s</o_gt>
        <d_gt>%s</d_gt>
        <imsi>%s</imsi>
        <password>123</password>
        <sc_number>%s</sc_number>
        <key_id>1</key_id>
        <command>1</command>
        <corr_id>1</corr_id>
        <seed>1122</seed>
        <msisdn>972559300036</msisdn>
        <message>%s</message>
        <sms_imsi>%s</sms_imsi>
        <map>3</map>
        </ss7gw>""" % (ogt, msc, imsi, ogt, payload, imsi)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

def sai(ogt, dgt, imsi, node):
    req = """<?xml version=\"1.0\"?>
          <ss7gw request=\"SAIN\">
          <d_ssn>6</d_ssn>
          <o_gt>%s</o_gt>
          <d_gt>%s</d_gt>
          <o_ssn>7</o_ssn>
          <imsi>%s</imsi>
          <num_req_vec>1</num_req_vec>
          <sccp_np>7</sccp_np>
          <node_type>%s</node_type>
          </ss7gw>""" % (ogt, dgt, imsi, node)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

def psi(ogt, dgt, imsi):
    req = """<?xml version=\"1.0\"?>
    <ss7gw request=\"PSI\">
    <o_ssn>6</o_ssn>
    <d_ssn>7</d_ssn>
    <o_gt>{0}</o_gt>
    <d_gt>{1}</d_gt>
    <imsi>{2}</imsi>
    </ss7gw>""".format(ogt, dgt, imsi)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

def psl(ogt, dgt, imsi):
    req = """<?xml version=\"1.0\"?>
    <ss7gw request=\"PSL\">
    <o_ssn>6</o_ssn>
    <d_ssn>7</d_ssn>
    <o_gt>{0}</o_gt>
    <d_gt>{1}</d_gt>
    <imsi>{2}</imsi>
    <location_type>1</location_type>
    </ss7gw>""".format(ogt, dgt, imsi)
    resp = executeHTTP(req, ss7_url, ss7_path)
    return resp

def air(imsi, ohost, orealm, drealm, mcc, mnc):
    req = """<?xml version=\"1.0\"?>
         <roam:air xmlns:roam="roamability:gtw:s6a">
         <roam:o-realm>%s</roam:o-realm>
         <roam:o-host>%s</roam:o-host>
         <roam:d-realm>%s</roam:d-realm>
         <roam:imsi>%s</roam:imsi>
         <roam:vplmn  mcc="%s" mnc="%s"/>
         <roam:eutran  numVec="1" immediate="1"/>
         </roam:air>""" % (orealm, ohost, drealm, imsi, mcc, mnc)
    resp = executeHTTP(req, S6A_URL, S6A_PATH)
    return resp

def clr(imsi, ohost, orealm, dhost, drealm):
    req = """<roam:clr proxy="true" xmlns:roam="roamability:gtw:s6a">
          <roam:o-realm>%s</roam:o-realm>
          <roam:o-host>%s</roam:o-host>
          <roam:d-realm>%s</roam:d-realm>
          <roam:d-host>%s</roam:d-host>
          <roam:imsi>%s</roam:imsi>
          <!--Optional:
          <roam:supp-feat vendor="3" list_id="2" list="7"/>-->
          <!--Optional:
          <roam:vendor-specific-app-id vendor-id="7" auth-app-id="7" acct-app-id="7"/>-->
          <!-- MME_UPDATE_PROCEDURE (0)-->
          <!-- SGSN_UPDATE_PROCEDURE (1)-->
          <!-- SUBSCRIPTION_WITHDRAWAL (2)-->
          <!-- UPDATE_PROCEDURE_IWF (3)-->
          <!-- INITIAL_ATTACH_PROCEDURE (4)-->
          <roam:cancel-type>2</roam:cancel-type>
        </roam:clr>""" % (orealm, ohost, drealm, dhost, imsi)
    resp = executeHTTP(req, S6A_URL, S6A_PATH)
    return resp

# imsi = 724540540423172
# ogt = 97254120624
# dgt = 550040540423172
node = 0
# soup = BeautifulSoup(
# #     psl('48507909001', '79639743780', '260036610000004')
# #     psi('97254120624', '447785014055', '425019610777082')
# #     sendSMS(ogt, imsi, msc, payload)
#     ,'xml')
# soup

In [1095]:
 BeautifulSoup(rb.sri4sm('48790993073', '447786205094'), 'xml')

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response>

In [494]:
reload(rb)
imsi = 260036610000000
# imsi = 260060169326039
orealm = 'nakamobile.mnc003.mcc260.3gppnetwork.org'
ohost = 'nakahss.nakamobile.mnc003.mcc260.3gppnetwork.org'

drealm = 'epc.mnc020.mcc234.3gppnetwork.org'
dhost = 'node245.epc.mnc020.mcc234.3gppnetwork.org'

mcc = '228'
mnc = '03'
soup = BeautifulSoup(
#     rb.air(imsi, ohost, orealm, drealm, mcc, mnc)
    rb.clr(imsi, ohost, orealm, dhost, drealm)
    ,'xml')
soup

<?xml version="1.0" encoding="utf-8"?>
<cla>
<session-id>44533bc64643b7ac88448d9e6369a05d18a0ad92ac950f630d207f95318ee3f2;1580214580;8978515</session-id>
<orig-host>deakat01.mnc003.mcc260.3gppnetwork.org</orig-host>
<orig-realm>epc.mnc020.mcc234.3gppnetwork.org</orig-realm>
<res-code>3002</res-code>
<error-msg>DRL-ERR-3002-312:.</error-msg>
</cla>

 ### Format output

In [ ]:
import datetime as dt
print( 'start: {} {} :end'.format('one', 'two') ) #start: one two :end
print( 'start: {1} {0} :end'.format('one', 'two') ) #start: two one :end
print( 'start: test {:10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:10} :end'.format(10) ) #start: test         10 :end
print( 'start: test {:>10} :end'.format('test') ) #start: test       test :end
print( 'start: test {:<10} :end'.format('test') ) #start: test test       :end
print( 'start: test {:_<10} :end'.format('test') ) #start: test test______ :end
print( 'start: test {:^10} :end'.format('test') ) #start: test    test    :end
print( 'start: test {:.10} :end'.format('test1test2test3') ) #start: test test1test2 :end
print( 'start: test {:_^10.5} :end'.format('test1test2test3') ) #start: test __test1___ :end
print( 'start: test {:d} :end'.format(42) ) #start: test 42 :end
print( 'start: test {:f} :end'.format(42) ) #start: test 42.000000 :end
print( 'start: test {:06.2f} :end'.format(3.141592) ) #start: test 003.14 :end
print( 'start: test {p[first]} {p[last]} :end'.format(p={'first': 'Jean-Luc', 'last': 'Picard'}) ) #start: test Jean-Luc Picard :end
print( 'start: test {p[4]} {p[5]} :end'.format(p=[4, 8, 15, 16, 23, 42]) ) #start: test 23 42 :end
print( 'start: test {:%Y-%m-%d %H:%M} :end'.format(dt.datetime(2001, 2, 3, 4, 5)) ) #start: test 2001-02-03 04:05 :end

### Search in files by Tags

In [8]:
import roamability as rb
reload(rb)

# Где искать файлы .txt с тегами
paths = ['c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_HOW_IT_WORKS/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/',
        'c:/W_DATA_ROAM/ПРОЕКТЫ/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/NOC_KNOWN_ISSUES/',
        'c:/W_DATA_ROAM/МЕТОДИКИ/_NOC_M_NEW_CONNECTION/',
        'c:/W_DATA_ROAM/ALARMS/',
        'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES',
        ]

# Если хотя бы один из тэгов присутствует. Нужно указать минимум один.
# any_tags  = ['#reports_bss_usage_mop']
# any_tags  = ['#gy_principle_of_operation']
# any_tags  = ['#ocs_principle_of_operation']
# any_tags  = ['#reports_ocs_usage_mop']
# any_tags  = ['#reports_ocs_subscribers']
# any_tags = ['#epg_principle_of_operation']
any_tags  = ['#sim_card_principle_of_operation']

# Если все теги присутствуют. Нужно оставить лист пустым [] если нет обязательных тэгов.
all_tags = []

rb.find_files_by_tags(paths, all_tags, any_tags)

****************************************************************************************************
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\SimApplet.txt
----------------------------------------------------------------------------------------------------
#sim_card_principle_of_operation
------------------------------
После апгрейда аплета до ver.7 нужно снова загружать IMSI
"when u upgrade the applet version , u remove all the provisioned (it will always happen with applet upgrading ) so there were no sponsor IMSI's in the applet and therefore the sim could not register.
this is why we need to use the late activation code to provision sponsor imsi and basic connectivity and after that everything went back to normal.
so from your side, u need to make sure that after upgrading any sim applet, if the sim exists and activated in Roamability OCS , then u MUST disconnect it , wait 10 minutues and re-activate it
if the sim does not exists yet on OCS portal , then u just need to activate

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 638: character maps to <undefined>

### Search in ALARMS by folder names

In [94]:
import os

paths = [
         r'c:/W_DATA_ROAM/ALARMS/',
         r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ/PRICES/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ/',
         r'c:/W_DATA_ROAM/ПРОЕКТЫ_NEW_CONNECTION/',
    r'c:/W_DATA_ROAM/ДОКУМЕНТАЦИЯ',
    r'c:\W_DATA_ROAM_DOC\ПРОЕКТЫ',
        ]

for path in paths:
    names = os.listdir(path)
    [print(normpath(path + name)) for name in names if name.upper().find('GBRVF')!=-1]

c:\W_DATA_ROAM\ALARMS\MOC_Partner_GBRVF_191010
c:\W_DATA_ROAM\ALARMS\MTC_Partner_GBRVF_191212
c:\W_DATA_ROAM\ALARMS\PS_P4_GBRVF_190328
c:\W_DATA_ROAM\ALARMS\PS_Partner_4G_issue_GBRVF_CHEOR_and_other_Vodafone_Group_191230
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\BREXIT_GBRME_GBRVF_GIBGT_200123.msg
c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\P4_GBRVF_MOC_wrong_charge.msg
c:\W_DATA_ROAM\ПРОЕКТЫ\PP_TOTTILI_GBRVF_BLOCK


### Search in Allowed Lists and others

In [89]:
downloads = 'C:/Users/balob/Downloads'
dmi_allowed_list = 'DMI.dmi_allowed_list_export_Thu_May_21_2020.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Tue_Apr_28_2020.csv'
bss_usage = 'bss_usage_2019_08.csv' # c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\NOC_HOW_IT_WORKS\BSS_SQL\Report_BSS_Usage_per_PLMN.txt

#Like expression for Allowed List
df_al=pd.read_csv(join(downloads, dmi_allowed_list))
df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace={r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*':'S1',
                                                        r'[Pp]4[\s\w-]*|^S2_[\s\w-]*':'S2',
                                                        r'[Mm][Bb][\s\w-]*|^S4_[\s\w-]*':'S4',
                                                        r'SMART[\s\w-]*|^S5_[\s\w-]*':'S5',
                                                        r'^S8_[\s\w-]*':'S8'}, regex=True)
df_al.rename({'PLMN code':'TADIG'}, axis=1, inplace=True)

df_usage = pd.read_csv(join(downloads, bss_usage), sep=';')
cols = ['REP_DATE','VPLMN','SPONSOR','MOC','MOC_SMS','DATA']
df_usage = df_usage.pivot_table(values='COST', index=['REP_DATE', 'VPLMN', 'SPONSOR'], columns='USAGE_TYPE').reset_index()[cols]
df_usage.rename({'VPLMN':'TADIG'}, axis=1, inplace=True)
df_usage.columns.name=''

df_al = pd.merge(df_al, df_usage, how='left', on=['SPONSOR','TADIG'])

df_np=pd.read_csv(join(downloads, dmi_netpfx))

In [17]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG']).SPONSOR.unique()

array(['S2', 'S1'], dtype=object)

In [103]:
sponsor = 'P4' # Use regular expressions if it's needed
tadig = 'GBRVF'

df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','REP_DATE','MOC','MOC_SMS','DATA','SPONSOR']].sort_values(by=['Sponsor','TADIG'])

,Sponsor,TADIG,REP_DATE,MOC,MOC_SMS,DATA,SPONSOR
6915,P4_Netmore_Comb,GBRVF,2019-08,94.590104,2.40361,242.598782,S2


In [147]:
df_al.loc[(df_al['TADIG'].str.startswith(tadig, na=False)) &
          (df_al['Sponsor'].str.contains(sponsor, na=False, regex=True, flags=re.I)),
          ['Sponsor','TADIG','SPONSOR']].sort_values(by=['Sponsor','TADIG']).Sponsor.unique()

array(['MB_DRIM01_ALL', 'P4_DRIM01_ALL', 'Partner_DRIM01_A',
       'S5_DRIM01_ALL'], dtype=object)

In [101]:
plmn = 'GBRME'
df1 = df_al.loc[(df_al['TADIG'].str.startswith(plmn, na=False)) &
          (df_al['Sponsor'].str.contains('', na=False)), ['Sponsor','TADIG']].sort_values(by=['Sponsor','TADIG'])
df2 = df_np[(df_np.Operator.str.startswith(plmn,na=False))] #.to_csv(join(downloads,'Prefix.csv')) # & (df_np.SSN==7)
display(df1)
display(df2)

,Sponsor,TADIG
6916,P4_Netmore_Comb,GBRME
2165,PA_Telzar_Soft,GBRME
136,Partner,GBRME
2680,Partner_All,GBRME
693,Partner_Combined,GBRME
5753,Partner_DENT01_C,GBRME
2432,Partner_Dialoq_A,GBRME
6453,Partner_FlexiCom,GBRME
3102,Partner_RedT_Sof,GBRME
5043,Partner_Tottolli,GBRME


,Operator,Prefix,Number/Range,SSN,MCC,MNC
1642,GBRME,447953,1,7,234,30
1643,GBRME,447953,1,149,234,30
1644,GBRME,447958,1,7,234,30
1645,GBRME,447958,1,149,234,30
1646,GBRME,4478921,1,7,234,30
1647,GBRME,4478921,1,149,234,30
1648,GBRME,4475896,1,7,234,30
1649,GBRME,4475896,1,149,234,30
1650,GBRME,447973,1,7,234,30
1651,GBRME,447973,1,149,234,30


In [277]:
str(df_np[(df_np.Prefix.str.startswith('6683',na=False))].Prefix.unique().tolist())
df_np[(df_np.Prefix.str.startswith('35999',na=False))]

,Operator,Prefix,Number/Range,SSN,MCC,MNC
732,BGRMT,359999320011,0,7,284,13
733,BGRMT,359999320031,0,6,284,13
734,BGRMT,359999329999,0,1,284,13


### Hex Bin Dec conversion

In [9]:
hex_int = 0x2b
dec_int = 10
bin_int = 0b1010
# Hex to dec
print('Hex to dec')
print(f'{hex_int:0>16d}')
# Hex to bin
print('Hex to bin')
print(f'{hex_int:0>16b}')
# Dec to hex
print('Dec to hex')
print(f'{dec_int:F>16X}')
# Dec to bin
print('Dec to bin')
print(f'{dec_int:0>16b}')
# Bin to hex
print('Bin to hex')
print(f'{bin_int:F>16X}')
# Bin to dec
print('Bin to dec')
print(f'{bin_int:0>16d}')

Hex to dec
0000000000000043
Hex to bin
0000000000101011
Dec to hex
FFFFFFFFFFFFFFFA
Dec to bin
0000000000001010
Bin to hex
FFFFFFFFFFFFFFFA
Bin to dec
0000000000000010


In [222]:
7-243-1
4-4-2

f'{bin(7)} - {bin(243)} - {bin(1)}'
#2.61.6

'0b111 - 0b11110011 - 0b1'

In [223]:
print(bin(10915))
a = '11111110011001'
# = '00000011001101'
print(int(a[-14:-11],2),'-',int(a[-11:-3],2),'-',int(a[-3:],2) )

0b10101010100011
7 - 243 - 1


### SIM Decoders

In [20]:
import codecs
codecs.encode(b"Smartel", "hex").upper()
spn = '547269706C6554656C' # 00547269706C6554656C9000 'TripleTel'
spn = '476C6F62616C3153494D' # 02476C6F62616C3153499000 'Global1SIM'
spn = '5469636F6D' # 005469636F6DFFFFFFFF9000 'Ticom'
spn = '006E6F722E6E6574'
bytearray.fromhex(spn).decode()

'\x00nor.net'

In [308]:
# IMSI coding
imsi = '2948310000209045'
imsi = '29040400001100009000' # IMSI Ticom 0829483100002090459000 284130000020954
def imsi_conv(imsi, code='code'):
    """Conver deconvert IMSI for perso. code='code' / 'decode'"""
    if code=='code':
        imsi = '9' + imsi
        return ''.join([imsi[i] for i in [1, 0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])
    elif code=='decode':
        return ''.join([imsi[i] for i in [0, 3, 2, 5, 4, 7, 6, 9, 8, 11, 10, 13, 12, 15, 14]])

imsi_conv(imsi, code='decode')

'240400000110000'

In [1108]:
# ANY decoder
value = '98791232039009010707' # ICCID Ticom 987912320390090107079000
value = '07916437716243F0' # SPN Ticom 359999320001 FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFE1FFFFFFFFFFFFFFFFFFFFFFFF0791539999230010FFFFFFFF0000A89000
value = '98922718012227815611'
def any_decoder(value):
    value_conv = ''
    for i in range(1, len(value), 2):
        value_conv += value[i: i + 1]
        value_conv += value[i - 1: i]
    return value_conv
any_decoder(value)

'89297281102272186511'

## Testing

In [99]:
import smpplib
import smpplib.gsm
client = None
#parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts(u'Hello world!')
parts, encoding_flag, msg_type_flag = smpplib.gsm.make_parts("""Test message""")

client = smpplib.client.Client('172.18.11.20', 2770)
# client = smpplib.client.Client('172.18.11.90', 2770) # Test
client.connect()
client.bind_transceiver(system_id='test', password='test') # ChangeMe for test
for part in parts:
    pdu = client.send_message(
        source_addr_ton=smpplib.consts.SMPP_TON_INTL,
#         source_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure it is a byte string, not unicode:
        source_addr='17783003587',
#         source_addr='Israel Post',
        dest_addr_ton=smpplib.consts.SMPP_TON_INTL,
        #dest_addr_npi=smpplib.consts.SMPP_NPI_ISDN,
        # Make sure thease two params are byte strings, not unicode:
#         destination_addr='16282710997',
        destination_addr='15145911524',
        short_message=part,
        data_coding=encoding_flag,
        esm_class=msg_type_flag,
        registered_delivery=True,)
    print(f'PDU SQN: {pdu.sequence}')
client.unbind()
client.unbind()
client.disconnect()

PDU SQN: 2


In [747]:
import requests
ip_addr = '192.168.1.44'
port = 80
url_01 = f'http://{ip_addr}:{port}/climate;sensor=1&readattempt=10&temperature=25.0&humidity=35.0'
url_02 = f'http://{ip_addr}:{port}/climate;sensor=4&readattempt=10&temperature=25.0&humidity=35.0'
url_03 = f'http://{ip_addr}:{port}/gas;sensor=2&sensorValue=666'
url_04 = f'http://{ip_addr}:{port}/motion;sensor=3'
for url in [url_01, url_02, url_03, url_04]:
    r = requests.get(url)
    print(r.status_code)
    print(r.text.strip())

200
"OK"
200
"OK"
200
"OK"
200
"OK"


In [15]:
# Data
sdrRate = 1.34588
charge = 0.295
chargedUnits = 6641698
unit = 1024 * 1014
(charge * sdrRate / chargedUnits) * unit

0.0620707714240545

In [109]:
# Sms
sdrRate = 1.37522
charge = 0.16527
chargedUnits = 1
unit = 1
(charge * sdrRate / chargedUnits) * unit

0.22728260940000003

In [981]:
# Voice
sdrRate = 1.37522
charge = 0
chargedUnits = 14
unit = 60
(charge * sdrRate / chargedUnits) * unit

0.0

In [1027]:
sql_srt="""
SELECT 
DISTINCT o.Name,did.Description,did.RangeStart, did.RangeEnd, did.Count
FROM [dbo].[DIDRanges] did
LEFT JOIN [dbo].[SponsorDIDRanges] sdid ON did.ID = sdid.DIDRangeID
LEFT JOIN [dbo].[Sponsors] s ON sdid.SponsorID = s.ID
LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
WHERE o.Name LIKE 'X2%';"""
with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
    df_did_rdb = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_did_rdb[['RangeStart','RangeEnd']] = df_did_rdb[['RangeStart','RangeEnd']].astype('str')
df_did_rdb.drop_duplicates(inplace=True)

def f(x):
    l = []
    if 'UK' in x.Description:
        l.append('UK')
    elif 'USA' in x.Description:
        l.append('USA')
    else:
        l.append('Unk')
#     if '200311' in x.Description:
#         l.append('NEW_200311')
#     elif '200303' in x.Description:
#         l.append('NEW_200303')
#     else:
#         l.append('OLD')
    if '200311' in x.Description:
        l.append('NEW')
    elif '200303' in x.Description:
        l.append('NEW')
    else:
        l.append('OLD')
    if 'drimsim' in x.Description.lower():
        l.append('DrimSim')
    else:
        l.append(x.Description.split(' ')[-1])
    return l

df_did_rdb[['Country', 'Type', 'Customer']] = DataFrame(df_did_rdb.apply(f, axis=1).tolist())
df_did_rdb.head(3)

,Name,Description,RangeStart,RangeEnd,Count,Country,Type,Customer
0,X2one,X2One USA 200303 DrimSim,16282710001,16282710001,1,USA,NEW,DrimSim
1,X2one,X2One USA 200303 DrimSim,16282710002,16282710002,1,USA,NEW,DrimSim
2,X2one,X2One USA 200303 DrimSim,16282710003,16282710003,1,USA,NEW,DrimSim


In [1028]:
df_did_rdb.groupby(['Name','Country','Customer','Type'], as_index=False)['Count'].sum().sort_values(by='Type')

,Name,Country,Customer,Type,Count
0,X2one,USA,DrimSim,NEW,6000
1,X2one,USA,FREE,NEW,997
2,X2one,USA,OrangeTest,NEW,1
3,X2one,USA,Pavel,NEW,1
4,X2one,USA,test,NEW,1


In [1026]:
df_did_rdb.loc[df_did_rdb.RangeStart == '16672506138']
# df_did_rdb.loc[df_did_rdb.Customer == 'test']

,Name,Description,RangeStart,RangeEnd,Count,Country,Type,Customer
0,X2one,X2One USA 200303 1OT,16672506138,16672506138,1,USA,NEW,1OT


In [987]:
df_did_rdb.loc[df_did_rdb.Type == 'OLD',
               ['Description','RangeStart','RangeEnd','Count']].to_csv(join(downloads, 'X2oneOldDid.csv'), index=False)

In [1022]:
# df = pd.read_csv(join(downloads, 'Roamability_Drimsim_5K_replacement_200311.csv'),
df = pd.read_csv(join(downloads, 'US_Order_1_1k.csv'),
                 header=None, names=['RangeStart'], dtype='str')
df['DS'] = 'YES'
df.head()

,RangeStart,DS
0,16282710001,YES
1,16282710002,YES
2,16282710003,YES
3,16282710004,YES
4,16282710005,YES


In [1025]:
df1 = pd.merge(df_did_rdb.loc[df_did_rdb.Customer == 'DrimSim'], df, how='right', on='RangeStart')
df1.count()
df1.loc[df1.Customer.isna()]

,Name,Description,RangeStart,RangeEnd,Count,Country,Type,Customer,DS
999,NaN,NaN,16672506138,NaN,NaN,NaN,NaN,NaN,YES


In [434]:
ota_payload = '20603D79FB432EAE63F082E734F3B46085'
l = len(ota_payload)/2
as_rp = '0D0001'
as_mo = '0D0021'
sms_payload = f'02700000{int(l+5):X}{as_rp}0000{ota_payload}'
print(sms_payload)

02700000160D0001000020603D79FB432EAE63F082E734F3B46085


In [487]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes
# Avoid Option 3
while True:
    try:
        key = DES3.adjust_key_parity(get_random_bytes(24))
        break
    except ValueError:
        pass

hex_str = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
key = bytes.fromhex(hex_str)    

cipher = DES3.new(key, DES3.MODE_CBC)
# plaintext = b'We are no longer the knights who say ni!'
plaintext = bytes.fromhex('4E3F0E5A000000010018BC') 
plaintext = bytes.fromhex('4E3F0E5A00000001') 
msg = cipher.iv + cipher.encrypt(plaintext)
s=''
for b in msg:
    s+=f'{b:X}'
s
# 20603D79FB432EAE63F082E734F3B46085

'4155A1741F3A507AC4CFC395C16D9795'

### OTA messages

In [ ]:
# Hex string to bytes
hex_str = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
byte_str = bytes.fromhex(hex_str)

In [571]:
# RB GetInfo example
rb.get_info

'027000001F0D00010000BFFF01000000000000102F978372C730834EF445172BD26B8C8F'

In [689]:
# Send a SMS
sms_payload = rb.get_info
sms_payload = 'A0C200004ED14C820283818B46400881556677887FF600112912000004350270000030150E192525000000010E0A8A0E1BD80CABB2C3F3903D80EF579BAEECBE6941A6DC0D437D553FE120026765CF497DEE5D'
# resp = sendSMS('97254120624', '425019613000536', '79112009927', sms_payload.upper()) # My test SIM
resp = sendSMS('550170002133', '724179999990002', '551184137200', sms_payload.upper())
BeautifulSoup(resp,'xml')

<?xml version="1.0" encoding="utf-8"?>
<response>
<result>-1</result>
<error>timeout</error>
<src_gt/>
</response>

In [586]:
# Decode response from SIM
s = BeautifulSoup(resp,'xml').response.response.text
rb.decode_payload_response(s)

This SIM card contains the following IMSIs:
Slot 0 425019613000536
Slot 1 260060149000536
Slot 2 234500026400001
Slot 3 425019617000000
Slot 4 260036610000004

Applet ver.: 1.8.0
MCCMNC: 25001


In [664]:
# Encrypt plaintext to OTA payload

import math
from Crypto.Cipher import DES3

# Roamability Test keys example

key_hex_str = 'D0AEE50D70087C19D773696FE698C78832432DF14F2487D1'
key = bytes.fromhex(key_hex_str)

# Plaintext

plaintext = '4E3F0E5A000000010018BC'
print(f'The original plaintext: {plaintext}')

# Split plaintext into 8 byte parts for DES3 CBC encryption

byte_array_list = []
for i in range(1, math.ceil(len(plaintext)/16)+1):
    hex_str = f'{plaintext[(i-1)*16:i*16]:0<16}'
    print(f'Plaintext 8 byte part {i}: {hex_str}')
    byte_array = bytes.fromhex(hex_str)
    byte_array_list.append(byte_array)

# Create cipher

cipher = DES3.new(key, DES3.MODE_CBC)

# Encrypt 8 byte parts

cipher_str = ''
for byte_array in byte_array_list:
    msg = cipher.encrypt(byte_array_list[0])
    for b in msg:
        cipher_str += f'{b:X}'
print(f'Encrypted:        {cipher_str}')
print('Original example: 2028D0FC11B4CD2E540704D7775B065DA8')

The original plaintext: 4E3F0E5A000000010018BC
Plaintext 8 byte part 1: 4E3F0E5A00000001
Plaintext 8 byte part 2: 0018BC0000000000
Encrypted:        3A2AEF7E5E182F38FCFFA3AF205C1A94
Original example: 2028D0FC11B4CD2E540704D7775B065DA8


In [675]:
checksum = 0
packet = bytes.fromhex('2028D0FC11B4CD2E540704D7775B065D')
for el in packet:
    checksum ^= el

print(hex(checksum))

0x95


In [685]:
# Make payload for SMS
ota_payload = '8010000016FBF7FF7F1F0E0FCB4100001FE3869804070000000000'

as_rp = '0D0001'
as_mo = '0D0021'
header = '0000BFFF010000000000'
l = (len(ota_payload) + len(as_rp) + len(header))/2
sms_payload = f'02700000{int(l):X}{as_rp}0000BFFF010000000000{ota_payload}'
print(sms_payload)

02700000280D00010000BFFF0100000000008010000016FBF7FF7F1F0E0FCB4100001FE3869804070000000000


In [1100]:
5000 * 0.22

1100.0

### Developments

In [161]:
import numpy as np
import pandas as pd
from pandas import DataFrame
df = DataFrame({'name':['one', 'two', 'three']*4, 'desc':['Hello', 'world']*6,
                'num1':np.random.randint(0, 100, size=12), 'num2':np.random.randint(0, 100, size=12)})
display(df)
display(df.groupby('name')['desc'].apply(lambda x: x.str.contains('ell').sum()))
display(df.groupby('name').apply(lambda df: sum(df['num1']+df['num2'])))

,name,desc,num1,num2
0,one,Hello,70,44
1,two,world,6,22
2,three,Hello,81,32
3,one,world,90,24
4,two,Hello,28,77
5,three,world,36,7
6,one,Hello,4,69
7,two,world,57,91
8,three,Hello,26,41
9,one,world,98,29


name
one      2
three    2
two      2
Name: desc, dtype: int64

name
one      428
three    360
two      407
dtype: int64

In [173]:
import pandas as pd
import datetime as dt
df = pd.DataFrame({'date':pd.date_range('3/9/2012 9:30', periods=5, freq='D'), 'desc':np.random.randint(0, 100, size=5)})
display(df)
df['date'].dt.day_name()

,date,desc
0,2012-03-09 09:30:00,54
1,2012-03-10 09:30:00,24
2,2012-03-11 09:30:00,5
3,2012-03-12 09:30:00,21
4,2012-03-13 09:30:00,37


0      Friday
1    Saturday
2      Sunday
3      Monday
4     Tuesday
Name: date, dtype: object

In [188]:
import pandas as pd
from pandas import Series, DataFrame
df=DataFrame(np.arange(200,250).reshape((10,5)),
             columns=['apple','orange','pear','plum','banana'],
             index=pd.Index(pd.date_range(start='1/1/2000 00:00:00', periods=10,freq='1D'), name='Date'))
df.to_csv('Test.csv')
display(df)
df = pd.read_csv('Test.csv',
#                  index_col=0, #index_col="Date"
                 dtype={'apple':'float','orange':'int','pear':'object'},
                 #header=0, #header=None если нет заголовков и они присваиваются автоматически
#                  names=['name','sex','births','plum','banana'], #задаются заголовки вручную
#                  skiprows=1, #сколько пропустить строк или можно указать конкретные строки skiprows=[0,3,5]. Если в файле есть заколовки, чтобы они не были прочитаны как данные указать skiprows=1.
                 na_values=[200, 210], #какие символы рассматривать как NaN
                 #comment='#', #один имвол, начинающий строку комментария
                 nrows=5,
                 parse_dates=['Date'], date_parser=lambda x: pd.datetime) #Количество читаемых строк от начала файла.
df

,apple,orange,pear,plum,banana
Date,,,,,
2000-01-01,200,201,202,203,204
2000-01-02,205,206,207,208,209
2000-01-03,210,211,212,213,214
2000-01-04,215,216,217,218,219
2000-01-05,220,221,222,223,224
2000-01-06,225,226,227,228,229
2000-01-07,230,231,232,233,234
2000-01-08,235,236,237,238,239
2000-01-09,240,241,242,243,244


,Date,apple,orange,pear,plum,banana
0,2000-01-01,NaN,201,202,203,204
1,2000-01-02,205.0,206,207,208,209
2,2000-01-03,NaN,211,212,213,214
3,2000-01-04,215.0,216,217,218,219
4,2000-01-05,220.0,221,222,223,224


In [157]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
df=DataFrame(np.arange(200,224).reshape((6,4)), columns=list('abcd'), index=['Utah','Ohio','Texas']*2)
display(df)
df.groupby(level=0).apply(lambda df, x, y: sum(df[x]+df[y]), 'a', 'b')
# df.groupby(level=0).apply(lambda df: sum(df['a']+df['b']))

,a,b,c,d
Utah,200,201,202,203
Ohio,204,205,206,207
Texas,208,209,210,211
Utah,212,213,214,215
Ohio,216,217,218,219
Texas,220,221,222,223


Ohio     842
Texas    858
Utah     826
dtype: int64

In [ ]:
%reset -f